In [1]:
#remove cell visibility
from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

In [2]:
%matplotlib inline
import control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

## Controllo della velocità longitudinale di un quadrotor

<img src="Images\EX33-QuadrotorLong.PNG" alt="drawing" width="300x300">

La velocità longitudinale di un quadrotor $v$ può essere controllata inclinando il veicolo sull'asse di beccheggio. L'angolo $\theta$ viene controllato applicando la coppia $T$ utilizzando le eliche. Il momento di inerzia del veicolo è $J= 1.3e-2$. Quando il veicolo è inclinato dell'angolo $\theta$, le eliche producono una forza in avanti approssimativamente uguale a $F_v = F\theta = mg\theta$ e la resistenza aerodinamica è $F_c=-cv=-0.9v$, dove $m=2000$ g è la massa del veicolo e $g = 9.81$ m/s^2 è l'accelerazione di gravità. La coppia massima è pari a $5000$ mNm. L'angolo di inclinazione $\theta$ deve essere limitato a $\pm30$ gradi durante tutte le operazioni mentre la velocità massima a 2 m/s. L'angolo di beccheggio viene stimato da un sensore appropriato e la velocità viene misurata con il GPS.

La procedura di progettazione è divisa in due fasi:
1. Scrivere le equazioni del sistema in forma di stato per la dinamica di rotazione (dalla coppia $T$ all'angolo di beccheggio $\theta$) e per la dinamica longitudinale (dall'angolo di beccheggio alla velocità di avanzamento $v$).
2. Progettare un regolatore per $v$ in modo da soddisfare le seguenti specifiche:
    - Tempo di assestamento al 5% inferiore a 2,5 secondi.
    - Nessun overshoot.
    - Nessun errore di regime in risposta a una richiesta di velocità a gradino.

### Equazioni del sistema

Le equazioni del sistema sono:

\begin{cases}
    m\dot{v} = F_v + F_c = mg\theta -cv \\
    J\ddot{\theta} = T.
\end{cases}

Definendo il vettore di stato come $x = \begin{bmatrix} x_1 & x_2 & x_3 \end{bmatrix}^T = \begin{bmatrix} v & \theta & \dot{\theta} \end{bmatrix}^T$ e l'input $u=T$, le equazioni nella forma di stato diventano:

\begin{cases}
\dot{x} = \begin{bmatrix} -c/m & g & 0 \\ 0 & 0 & 1 \\ 0 & 0 & 0 \end{bmatrix}x + \begin{bmatrix} 0 \\ 0 \\ 1/J \end{bmatrix}u \\
y = \begin{bmatrix} y_1 \\ y_2 \end{bmatrix} = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \end{bmatrix}x
\end{cases}

La dinamica da $u$ a $\theta$ è un doppio integratore mentre quella da $\theta$ a $v$ è una dinamica del primo ordine con un polo in $-c/m$. Il sistema ha due uscite: velocità e angolo di inclinazione.

La matrice di controllabilità $\mathcal{C}$ è

In [3]:
A = numpy.matrix('-0.45 9.81 0; 0 0 1; 0 0 0')
B = numpy.matrix([[0],[0],[1/1.3E-02]])
C = numpy.matrix('1 0 0; 0 1 0')
D = numpy.matrix('0; 0')

CM = control.ctrb(A,B)
display(Markdown(bmatrix(CM)))
# print(numpy.linalg.matrix_rank(CM))

\begin{bmatrix}
  0. & 0. & 754.61538462\\
  0. & 76.92307692 & 0.\\
  76.92307692 & 0. & 0.\\
\end{bmatrix}

e ha rango pari a 3 quindi il sistema è controllabile.
La matrice di osservabilità $\mathcal{O}$ è

In [4]:
OM = control.obsv(A,C)
display(Markdown(bmatrix(OM)))
# print(numpy.linalg.matrix_rank(OM))

\begin{bmatrix}
  1. & 0. & 0.\\
  0. & 1. & 0.\\
  -0.45 & 9.81 & 0.\\
  0. & 0. & 1.\\
  0.2025 & -4.4145 & 9.81\\
  0. & 0. & 0.\\
\end{bmatrix}

e ha rango pari a 3 quindi il sistema è osservabile.

### Design del regolatore
#### Design dell'osservatore
Poiché si hanno le misure dirette di $x_1$ e $x_2$, si è interessati a stimare solamente $x_3$. Se si guarda il sottosistema $(x_2, \, x_3)$ si nota che questo è osservabile, quindi è possibile progettare un osservatore considerando solo questo sottosistema. La struttura dello stimatore è quindi:

$$
\begin{bmatrix} \dot{\hat{x}_2} \\ \dot{\hat{x}_3} \end{bmatrix} = \begin{bmatrix} 0 & 1 \\ 0 & 0 \end{bmatrix}\begin{bmatrix} \hat{x}_2 \\ \hat{x}_3 \end{bmatrix} + \begin{bmatrix} 0 \\ 1/J \end{bmatrix}u + \begin{bmatrix} l_1 \\ l_2 \end{bmatrix}\left( y - C\begin{bmatrix} \hat{x}_2 \\ \hat{x}_3 \end{bmatrix} \right) = \begin{bmatrix} -l_1 & 1 \\ -l_2 & 0 \end{bmatrix}\begin{bmatrix} \hat{x}_2 \\ \hat{x}_3 \end{bmatrix} + \begin{bmatrix} 0 \\ 1/J \end{bmatrix}u + \begin{bmatrix} l_1 \\ l_2 \end{bmatrix}y
$$

applicando la trasformata di Laplace e risolvendo per $\hat{x}_3(s)$ si arriva a

$$
\hat{x}_3(s) = \frac{l_2s}{s^2+l_1s+l_2}y_2(s) + \frac{s+l_1}{s^2+l_1s+l_2}\frac{u(s)}{J}.
$$

Ora si ha un semplice stimatore lineare per $x_3$ che è asintoticamente stabile per qualsiasi $l_1>0$ e $l_2>0$. È interessante notare che se $l_2\rightarrow \infty$, la funzione di trasferimento dello stimatore si semplifica in $\hat{x}_3(s) = s y_2(s)$ e il risultato è uguale a $\hat{x}_3 = \dot{\theta}$ che quindi si ottiene semplicemente differenziando l'uscita $y_2 = \theta$ misurata.

La scelta di $l_1 = 20$ e $l_2 = 100$ permette di avere entrambi gli autovalori dell'osservatore in $-10$.

#### Design del controller
Per il requisito del tempo di assestamento, la frequenza dei poli deve essere maggiore di $3/T_s$ per i poli reali e maggiore di $3/\zeta T_s$ per i poli complessi, dove $T_s$ è il tempo di assestamento (5%) e $\zeta$ lo smorzamento. Una buona posizione dei poli, in termini di risposta e di energia in ingresso, per un sistema a doppio integratore, dovrebbe rientrare in un intervallo di $\pm 45°$ rispetto all'asse reale negativo. Considerando prima questi fatti e poi procedendo iterativamente, i poli dominanti sono stati scelti in $-2.8\pm1.0i$, mentre il terzo polo è stato scelto con una frequenza molto più alta: $-15$.
Per la richiesta di errore stazionario nullo, l'ingresso di riferimento viene scalato di un guadagno uguale all'inverso del guadagno statico del sistema a ciclo chiuso, ottenendo un guadagno ad anello chiuso totale pari a $1.0$.

### Come usare questo notebook?
- Verifica le specifiche richieste in caso di errore iniziale nella stima di $x_3$; sia per errore positivo che negativo.
- Guarda la risposta modificata e, avendo in mente un sistema fisico, cerca di capire perché è cambiata in quel modo.

In [5]:
# Preparatory cell

X0 = 0.0
K = numpy.matrix([8/15,-4.4,-4])
L = numpy.matrix([[66],[107/3]])

Aw = matrixWidget(3,3)
Aw.setM(A)
Bw = matrixWidget(3,1)
Bw.setM(B)
Cw = matrixWidget(1,3)
Cw.setM(C)
X0w = widgets.FloatText(
    value=X0,
    description='',
    disabled=False
)
Kw = matrixWidget(1,3)
Kw.setM(K)
Lw = matrixWidget(2,1)
Lw.setM(L)


eig1c = matrixWidget(1,1)
eig2c = matrixWidget(2,1)
eig3c = matrixWidget(1,1)
eig1c.setM(numpy.matrix([-15.])) 
eig2c.setM(numpy.matrix([[-2.8],[-1.0]]))
eig3c.setM(numpy.matrix([-15.]))

eig2o = matrixWidget(2,1)
eig3o = matrixWidget(1,1)
eig2o.setM(numpy.matrix([[-10.],[0.]]))
eig3o.setM(numpy.matrix([-10.]))

In [6]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

# Define type of method 
selm = widgets.Dropdown(
    options= [('Imposta K','Set K'), ('Imposta gli autovalori','Set the eigenvalues')],
    value= 'Set the eigenvalues',
    description='',
    disabled=False
)

# Define the number of complex eigenvalues
sele = widgets.Dropdown(
    options= [('0 autovalori complessi','0 complex eigenvalues'), ('2 autovalori complessi','2 complex eigenvalues')],
    value= '2 complex eigenvalues',
    description='Autovalori complessi:',
    style = {'description_width': 'initial'},
    disabled=False
)

#define type of ipout 
selu = widgets.Dropdown(
    options=[('impulso','impulse'), ('gradino','step'), ('sinusoide','sinusoid'), ('onda quadra','square wave')],
    value='step',
    description='Riferimento:',
    style = {'description_width': 'initial'},
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=2,
    min=0,
    max=4,
    step=0.1,
    description='Riferimento:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=0.5,
    min=0.001,
    max=10,
    step=0.001,
    description='Periodo: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

gain_w2 = widgets.FloatText(
    value=1.,
    description='Guadagno inverso del riferimento:',
    style = {'description_width': 'initial'},
    disabled=True
)

simTime = widgets.FloatText(
    value=3,
    description='Tempo di simulazione (s):',
    style = {'description_width': 'initial'},
    disabled=False
)

In [7]:
# Support functions

def eigen_choice(sele):
    if sele == '0 complex eigenvalues':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = True
        eig2o.children[1].children[0].disabled = True
        eig = 0
    if sele == '2 complex eigenvalues':
        eig1c.children[0].children[0].disabled = True
        eig2c.children[1].children[0].disabled = False
        eig2o.children[1].children[0].disabled = False
        eig = 2
    return eig

def method_choice(selm):
    if selm == 'Set K and L':
        method = 1
        sele.disabled = True
    if selm == 'Set the eigenvalues':
        method = 2
        sele.disabled = False
    return method

In [8]:
# Reduced system
Ar = numpy.matrix('0 1; 0 0')
Br = numpy.matrix([[0],[1/1.3E-02]])
Cr = numpy.matrix('1 0')
Dr = numpy.matrix('0')

def main_callback2(Aw, Bw, X0w, K, L, eig1c, eig2c, eig3c, eig2o, eig3o, u, period, selm, sele, selu, simTime, DW):
    eige = eigen_choice(sele)
    method = method_choice(selm)
    
    if method == 1:
        solc = numpy.linalg.eig(A-B*K)
        solo = numpy.linalg.eig(Ar-L*Cr)
    if method == 2:
        if eige == 0:
            K = control.acker(A, B, [eig1c[0,0], eig2c[0,0], eig3c[0,0]])
            Kw.setM(K)
            
            L = control.acker(Ar.T, Cr.T, [eig2o[0,0], eig3o[0,0]]).T
            Lw.setM(L)
        if eige == 2:
            K = control.acker(A, B, [eig3c[0,0], 
                                     numpy.complex(eig2c[0,0],eig2c[1,0]), 
                                     numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
            
            L = control.acker(Ar.T, Cr.T, [numpy.complex(eig2o[0,0],eig2o[1,0]), 
                                           numpy.complex(eig2o[0,0],-eig2o[1,0])]).T
            Lw.setM(L)
            
    
    sys = control.ss(A,B,numpy.vstack((C,numpy.zeros((B.shape[1],C.shape[1])))),numpy.vstack((D,numpy.eye(B.shape[1]))))
    sysC = control.ss(numpy.zeros((1,1)),
                      numpy.zeros((1,numpy.shape(A)[0])),
                      numpy.zeros((numpy.shape(B)[1],1)),
                      -K)
    
    sysE = control.ss(Ar-L*Cr,
                      numpy.hstack((L,Br-L*Dr)),
                      numpy.matrix('0 1'),
                      numpy.zeros((1,2)))
    
    sys_append = control.append(sys, sysE, sysC, control.ss(A,B,numpy.eye(A.shape[0]),numpy.zeros((A.shape[0],B.shape[1]))))
    Q = []
    # y in ingresso a sysE
    for i in range(1):
        Q.append([B.shape[1]+i+1, i+2])
    # u in ingresso a sysE
    for i in range(B.shape[1]):
        Q.append([B.shape[1]+1+i+1, C.shape[0]+i+1])
    # u in ingresso a sys
    for i in range(B.shape[1]):
        Q.append([i+1, C.shape[0]+B.shape[1]+1+i+1])
    # u in ingresso al sistema che ha come uscite gli stati reali
    for i in range(B.shape[1]):
        Q.append([2*B.shape[1]+1+A.shape[0]+i+1, C.shape[0]+i+1])
    # xe in ingresso a sysC
    Q.append([2*B.shape[1]+1+1, 1])
    Q.append([2*B.shape[1]+1+1+1, 1+1])
    Q.append([2*B.shape[1]+1+2+1, C.shape[0]+B.shape[1]+1])
        
    inputv = [i+1 for i in range(B.shape[1])]
    outputv = [i+1 for i in range(numpy.shape(sys_append.C)[0])]
    sys_CL = control.connect(sys_append,
                             Q,
                             inputv,
                             outputv)
    
    t = numpy.linspace(0, 100000, 2)
    t, yout = control.step_response(sys_CL[0,0],T=t)
    dcgain = yout[-1]
    gain_w2.value = dcgain
    if dcgain != 0:
        u1 = u/gain_w2.value
    else:
        print('Il guadagno impostato per il riferimento è 0 e quindi viene cambiato a 1')
        u1 = u/1
    print('Il guadagno statico del sistema in anello chiuso (dal riferimento all\'uscita) è: %.5f' %dcgain)
    
    X0w1 = numpy.zeros((2*A.shape[0]+2,1))
    X0w1[A.shape[0]+1,0] = X0w
    if simTime != 0:
        T = numpy.linspace(0, simTime, 10000)
    else:
        T = numpy.linspace(0, 1, 10000)
      
    if selu == 'impulse': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        U1 = [0 for t in range(0,len(T))]
        U1[0] = u1
        T, yout, xout = control.forced_response(sys_CL,T,U1,X0w1)
    if selu == 'step':
        U = [u for t in range(0,len(T))]
        U1 = [u1 for t in range(0,len(T))]
        T, yout, xout = control.forced_response(sys_CL,T,U1,X0w1)
    if selu == 'sinusoid':
        U = u*numpy.sin(2*numpy.pi/period*T)
        U1 = u1*numpy.sin(2*numpy.pi/period*T)
        T, yout, xout = control.forced_response(sys_CL,T,U1,X0w1)
    if selu == 'square wave':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        U1 = u1*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, yout, xout = control.forced_response(sys_CL,T,U1,X0w1)
    # N.B. i primi 3 stati di xout sono quelli del sistema, mentre gli ultimi 3 sono quelli dell'osservatore
    
    step_info_dict = control.step_info(sys_CL[0,0],SettlingTimeThreshold=0.05,T=T)
    print('Step info: \n\tTempo di salita =',step_info_dict['RiseTime'],'\n\tTempo di assestamento (5%) =',step_info_dict['SettlingTime'],'\n\tOvershoot (%)=',step_info_dict['Overshoot'])
    print('Massimo valore di x2 (%)=', max(abs(yout[C.shape[0]+2*B.shape[1]+1+1]))/(numpy.pi/180*30)*100)
    
    fig = plt.figure(num='Simulation1', figsize=(14,12))
    
    fig.add_subplot(221)
    plt.title('Risposta dell\'uscita')
    plt.ylabel('Uscita')
    plt.plot(T,yout[0],T,U,'r--')
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$y$','Riferimento'])
    plt.grid()
    
    fig.add_subplot(222)
    plt.title('Ingresso')
    plt.ylabel('$u$')
    plt.plot(T,yout[C.shape[0]])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(223)
    plt.title('Risposta degli stati')
    plt.ylabel('Stati')
    plt.plot(T,yout[C.shape[0]+2*B.shape[1]+1],
             T,yout[C.shape[0]+2*B.shape[1]+1+1],
             T,yout[C.shape[0]+2*B.shape[1]+1+2],
             T,[numpy.pi/180*30 for i in range(len(T))],'r--',
             T,[-numpy.pi/180*30 for i in range(len(T))],'r--')
    plt.xlabel('$t$ [s]')
    plt.legend(['$x_{1}$','$x_{2}$','$x_{3}$','limite +$x_{2}$','limite -$x_{2}$'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(224)
    plt.title('Errore di stima')
    plt.ylabel('Errore')
    plt.plot(T,yout[C.shape[0]+2*B.shape[1]+1+2]-yout[C.shape[0]+B.shape[1]])
    plt.xlabel('$t$ [s]')
    plt.legend(['$e_{3}$'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    #plt.tight_layout()
   
alltogether2 = widgets.VBox([widgets.HBox([selm, 
                                          sele,
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('K:',border=3), Kw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Autovalori:',border=3),
                                          eig1c, 
                                          eig2c, 
                                          eig3c,
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3)]),
                            widgets.Label('X0 stim.:',border=3), X0w,
                            widgets.Label(' ',border=3), 
                            widgets.HBox([widgets.Label('L:',border=3), Lw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Autovalori:',border=3), 
                                          eig2o, 
                                          eig3o,
                                          widgets.Label(' ',border=3),
#                                           widgets.VBox([widgets.Label('Inverse reference gain:',border=3),
#                                                         widgets.Label('Simulation time (s):',border=3)]),
                                          widgets.VBox([gain_w2,simTime])]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([u, 
                                          period, 
                                          START])])
out2 = widgets.interactive_output(main_callback2, {'Aw':Aw, 'Bw':Bw, 'X0w':X0w, 'K':Kw, 'L':Lw,
                                                 'eig1c':eig1c, 'eig2c':eig2c, 'eig3c':eig3c, 'eig2o':eig2o, 'eig3o':eig3o, 
                                                 'u':u, 'period':period, 'selm':selm, 'sele':sele, 'selu':selu, 'simTime':simTime, 'DW':DW})
out2.layout.height = '870px'
display(out2, alltogether2)

Output(layout=Layout(height='870px'))